In [1]:
import os

In [2]:

dataset_dir = '../data/2024-10-30-loc-dataset-1024'
subset_dir = 'val'

image_name = 'tile_tif_13_11.png'
mask_name = 'tile_shp_13_11.png'

image_path = os.path.join(dataset_dir,subset_dir,'images',image_name)
mask_path = os.path.join(dataset_dir,subset_dir,'masks',mask_name)

In [4]:
import cv2
import numpy as np

# Load image and mask (assuming they are in grayscale or binary for the mask)
image = cv2.imread(image_path)  # natural image
mask = cv2.imread(mask_path, 0)  # mask in grayscale

# Step 1: Resize image with bicubic interpolation
image_resized = cv2.resize(image, (512, 512), interpolation=cv2.INTER_CUBIC)

# Step 2: Resize mask with nearest-neighbor interpolation
mask_resized = cv2.resize(mask, (512, 512), interpolation=cv2.INTER_NEAREST)

## Step 3: Optional - Enhance edges on the mask
#kernel = np.ones((3, 3), np.uint8)
#mask_dilated = cv2.dilate(mask_resized, kernel, iterations=100)

# Step 4: Optional - Apply CLAHE to the image for contrast enhancement
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
if len(image_resized.shape) == 3:  # if color image
    # Apply CLAHE on each channel
    image_resized = cv2.merge([clahe.apply(channel) for channel in cv2.split(image_resized)])
else:
    # Grayscale image
    image_resized = clahe.apply(image_resized)

# Save or use the processed image and mask
cv2.imwrite('00_image_resized.png', image_resized)
cv2.imwrite('00_mask_resized.png', mask_resized)


True

In [ ]:
def shadow_boosting():
    image = cv2.imread(image_path)  # natural image
    mask = cv2.imread(mask_path, 0)  # mask in grayscale

    # Step 1: Resize image with bicubic interpolation
    image_resized = cv2.resize(image, (512, 512), interpolation=cv2.INTER_CUBIC)

    # Step 2: Resize mask with nearest-neighbor interpolation
    mask_resized = cv2.resize(mask, (512, 512), interpolation=cv2.INTER_NEAREST)

    ## Step 3: Optional - Enhance edges on the mask
    #kernel = np.ones((3, 3), np.uint8)
    #mask_dilated = cv2.dilate(mask_resized, kernel, iterations=100)

    # Step 4: Optional - Apply CLAHE to the image for contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    if len(image_resized.shape) == 3:  # if color image
        # Apply CLAHE on each channel
        image_resized = cv2.merge([clahe.apply(channel) for channel in cv2.split(image_resized)])
    else:
        # Grayscale image
        image_resized = clahe.apply(image_resized)

### **Vegetation Index Calculation (e.g., NDVI)**
   - Calculate a **vegetation index** like the **Normalized Difference Vegetation Index (NDVI)** to highlight vegetation areas. This method requires an infrared (IR) channel, but if you have only RGB, you can approximate NDVI with a simplified green and red band calculation.
   - \( \text{NDVI} \approx \frac{\text{Green} - \text{Red}}{\text{Green} + \text{Red}} \)
   

In [47]:
import cv2
import numpy as np

# Load the image
image = cv2.imread(image_path)



# Convert to float for NDVI calculation
green = image[:, :, 1].astype(float)
red = image[:, :, 2].astype(float)

# Calculate NDVI
ndvi = (green - red) / (green + red + 1e-5)  # Adding small value to avoid division by zero

# Normalize NDVI to [0, 255] for better visibility
ndvi_normalized = cv2.normalize(ndvi, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

ndvi_normalized= cv2.resize(ndvi_normalized, (512, 512), interpolation=cv2.INTER_CUBIC)
# Save the result
cv2.imwrite('00_image_resized.png', ndvi_normalized)


True

### **Selective Color Desaturation (Remove Non-Green Colors)**
   - To emphasize green areas more, **desaturate or darken non-green colors** in the image. This will keep green areas (vegetation) prominent while reducing the impact of other colors.


In [5]:
import cv2
import numpy as np

# Load the image
image = cv2.imread(image_path)

# Convert to HSV color space
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define a mask for green colors
lower_green = np.array([35, 40, 40])  # Adjust these values as needed
upper_green = np.array([85, 255, 255])
green_mask = cv2.inRange(hsv, lower_green, upper_green)

# Desaturate areas outside of the green mask
hsv[:, :, 1] = cv2.bitwise_and(hsv[:, :, 1], green_mask)

# Convert back to BGR color space
desaturated_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

# Save the result
cv2.imwrite('00_image_resized.png', desaturated_image)


True